In [1]:
import numpy as np
import pandas as pd
from collections import Counter



In [2]:
item_list = pd.read_csv('movies.csv')
transaction_list = pd.read_csv('ratings.csv')

## Item Feature Table

In [3]:
def create_item_feature_table(item_list, encode_genre=False):
    """
    Creates item feature table from the movies.csv file
    Input:
    
    item_list (pd.DataFrame)      : MovieLens movies.csv Datraframe containing movieId, title, and genre
    encode_genre (bool)           : Include 1-hot encoding of movie into its indicated genres (default: False)

    
    Output
    item_feature_table : Dataframe containing  itemId, movieId, title, genres, year, genre 1-hot encoding (optional) 
    """
    
    item_table = item_list.copy()
    item_table.index.rename('item_id', inplace=True)
    item_table['year'] = item_table['title'].str.slice(-5,-1)
    item_table['genres'] = item_table['genres'].str.split('|')
    
    item_feature_table = item_table
    
    if (encode_genre):
        genre_list = list(set().union(*item_table['genres']))
        genre_list.sort()
        item_feature_table[genre_list] = 0
        
        def set_genres(row):
            row[row['genres']] = 1
            return row
        
        item_feature_table = item_feature_table.apply(set_genres, axis=1)
    
    return item_feature_table
    
    

In [4]:
item_feature = create_item_feature_table(item_list, encode_genre=True)
item_feature.head()

,movieId,title,genres,year,(no genres listed),Action,Adventure,Animation,Children,Comedy,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
item_id,,,,,,,,,,,,,,,,,,,,,
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",1995,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]",1995,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),"[Comedy, Romance]",1995,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]",1995,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
4,5,Father of the Bride Part II (1995),[Comedy],1995,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


## Transaction Table

In [5]:
def create_transaction_table(transaction_list, item_list, include_timestamp=False):
    """
    Creates user feature table from the ratings.csv file
    Input:
    
    transaction_list (pd.DataFrame)     : MovieLens movies.csv Datraframe containing movieId, title, and genre
    item_list (pd.DataFrame)            : MovieLens movies.csv Datraframe containing movieId, title, and genre
    include_timestamp (bool)            : Include timestamp in output 
    
    Output
    user_feature_table : Dataframe containing  user_id, item_id, rating 
    """
    
    user_table = transaction_list.copy()
    user_table.rename(columns = {'userId':'user_id'}, inplace=True)
    
    # create new index based on zero
    user_table['user_id'] = user_table.groupby('user_id').ngroup()
    user_table = user_table.set_index('user_id')

    user_table['item_id'] = 0
    
    item_feature = create_item_feature_table(item_list)
    
    def get_item_id(row):
        row['item_id'] =  int(item_feature.index[item_feature['movieId'] == row['movieId']][0])
        return row
        
    user_table = user_table.apply(get_item_id, axis=1)
    user_table['item_id'] = user_table['item_id'].astype(int)
    
    # option to include/remove timestamp column
    if not include_timestamp:
        del user_table['timestamp']
    
    # remove movieId column
    del user_table['movieId']
    
    return user_table
    
    

In [6]:
transaction_table = create_transaction_table(transaction_list, item_list)
transaction_table.head()

,rating,item_id
user_id,,
0,4.0,0
0,4.0,2
0,4.0,5
0,5.0,43
0,5.0,46


## User Feature Table

In [7]:
def create_user_feature_table(transaction_list, item_list, rating=0):
    """
    Counts the genres of the movies that the user has seen. Each movie can have several different genres
    Possible genres include: 'IMAX', 'Adventure', 'Mystery', 'Animation', 'Documentary', 'Comedy',
       'Western', 'War', 'Film-Noir', 'Crime', 'Drama', 'Thriller', 'Fantasy',
       'Action', 'Sci-Fi', 'Children', 'Romance', 'Horror', 'Musical',
       '(no genres listed)'
       
    Input:
    
    transaction_list (pd.DataFrame)     : MovieLens movies.csv Datraframe containing movieId, title, and genre
    item_list (pd.DataFrame)            : MovieLens movies.csv Datraframe containing movieId, title, and genre
    rating     : Filter movies with ratings better or equal this user-defined input (default = 0)
    
    Output
    user_feature : Dataframe containing userId, item_id, rating
    """
    
    item_feature = create_item_feature_table(item_list)
    genre_list = list(set().union(*item_feature['genres']))
    genre_list.sort()
    
    transaction_table = create_transaction_table(transaction_list, item_list)
    
    user_id_list = list(set(transaction_table.index))
    user_feature = pd.DataFrame({'user_id': user_id_list})
    user_feature = user_feature.set_index('user_id')
    user_feature[genre_list] = 0
    
    # for user-defined rating, filter dataframe based on input-rating
    if rating > 0:
        if rating > 5.0:
            rating = 5.0
        transaction_table = transaction_table[transaction_table['rating'] >= rating]

    user_id_list = list(set(transaction_table.index))
    user_item_list = transaction_table.groupby('user_id')['item_id'].apply(list)
#     print(user_item_list.iloc[441])
#     print(item_feature.iloc)
    for user in user_id_list:
        temp = list(item_feature.iloc[user_item_list[user]]['genres'])
        user_genre_list = [item for sublist in temp for item in sublist]
        user_genre_dict = Counter(user_genre_list)

        genre_list = user_genre_dict.keys()
        genre_list_count = user_genre_dict.values()
        user_feature.loc[user, genre_list] = genre_list_count
        

    return user_feature
    
    

In [8]:
user_feature = create_user_feature_table(transaction_list, item_list, rating=4)
user_feature.head()

,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
user_id,,,,,,,,,,,,,,,,,,,,
0,0.0,76.0,74.0,27.0,37.0,70.0,39.0,0.0,64.0,41.0,1.0,9.0,0.0,20.0,13.0,24.0,32.0,43.0,20.0,6.0
1,0.0,7.0,2.0,0.0,0.0,5.0,6.0,2.0,11.0,0.0,0.0,0.0,2.0,0.0,2.0,1.0,2.0,6.0,1.0,0.0
2,0.0,9.0,4.0,0.0,0.0,1.0,0.0,0.0,1.0,2.0,0.0,8.0,0.0,0.0,1.0,0.0,12.0,6.0,0.0,0.0
3,0.0,13.0,19.0,4.0,7.0,58.0,18.0,1.0,70.0,13.0,3.0,4.0,0.0,12.0,14.0,29.0,5.0,22.0,5.0,7.0
4,0.0,3.0,3.0,5.0,7.0,7.0,7.0,0.0,14.0,5.0,0.0,0.0,1.0,4.0,1.0,3.0,0.0,5.0,2.0,1.0
